In [2]:
import json
import jieba
import re
import numpy as np
import pandas as pd
from zhconv import convert
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

In [3]:
# 这是用来生成lsa向量的训练集，必须用这个数据集来拼接lsa向量
train_data_lsa_cvjd_54339 = pd.read_pickle('../generalization_data/train_data_lsa_cvjd_54339.pkl')
train_data_lsa_cvjd_54339

,cv_id,jd_id,jd_code,title,deadline,category_id,category_name,company_id,company_name,recruit_number,...,workYear,avatar,eduTracks,jobTracks,projectTracks,ds.1,da_label,rank_label,resume_type,order_month
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,JD20210910115615503390,市场营销负责人,2021-11-30 23:59:59.0,140101,市场营销,2c9207157bcbb76b017bcdd5f254038a,深圳市快易典教育科技有限公司,1,...,13.0,NaN,"['{""degree"": ""本科"", ""endDate"": ""2008-07"", ""id"":...","['{""companyName"": ""深圳市亿科数字科技有限公司"", ""descriptio...","['{""companyName"": ""*深圳市蜂联科技有限公司（奇虎360）"", ""desc...",20220831,3,3,NaN,2021-09
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,JD20211126124504034196,海外区域运营经理/高级经理,2022-02-28 23:59:59.0,130000,运营,2c9207157bf94d99017c1714f34e1715,广州市百果园网络科技有限公司,1,...,3.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2019-06"", ""id"":...","['{""companyName"": ""北京星制科技有限公司（全民快乐）"", ""descrip...","['{""description"": ""主播表现，直播平均 2 场，平台新用户 12926，主...",20220831,-1,2,4.0,2021-12
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,JD20211130113932574529,高级项目经理,NaN,310000,高级管理,2c9207157bf94d99017c2f902e952262,深圳纳欣科技有限公司,1,...,5.0,https://image0.lietou-static.com/big/5f8fa3bde...,"['{""degree"": ""硕士"", ""endDate"": ""2016-07"", ""id"":...","['{""companyName"": ""酷狗音乐"", ""companyTags"": ""互联网+...","['{""companyName"": ""酷狗音乐"", ""description"": ""酷狗20...",20220831,1,1,4.0,2021-12
5,2c9207157dbedf8d017dc1503c891402,2c9207157dc878e5017de5ee152b646c,JD20211223141525111469,阿里云智能-金融行业解决方案架构师（公有云）,NaN,100405,IT技术支持,2c9207157ccbcd86017d098f2b6454be,阿里云计算有限公司,1,...,11.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2010-07"", ""id"":...","['{""companyName"": ""华为技术有限公司"", ""department"": ""年...","['{""description"": ""项目职务: 项目经理\\n项目描述: 引导公司 IT ...",20220831,2,2,4.0,2021-12
6,2c9207157dbedf8d017dc1503c891402,2c9207157dbedfdb017dc14a85ff432f,JD20211216113026318551,金融行业解决方案架构师,NaN,101201,售前技术支持,2c9207157ccbcd86017d098f2b6454be,阿里云计算有限公司,2,...,11.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2010-07"", ""id"":...","['{""companyName"": ""华为技术有限公司"", ""department"": ""年...","['{""description"": ""项目职务: 项目经理\\n项目描述: 引导公司 IT ...",20220831,1,1,4.0,2021-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82315,8a69d7a68219933d018248864f066edd,8a69f601823a92b00182494187c01920,JD20220725145055775181,C++开发工程师-智能创作,NaN,100102,C++,2c9207157d95bfde017da1e665a9550d,北京字节跳动科技有限公司,1,...,9.0,NaN,"['{""schoolTag"": ""211"", ""endDate"": ""2022-01"", ""...","['{""endDate"": ""2022-07"", ""companyName"": ""北京开课吧...","['{""description"": ""项目说明: 短信平台是开课吧公司级短信需求处理的基础服...",20220831,2,2,4.0,2022-07
82316,8a69d7a68219933d01824944cb1533be,8a69c4a8820f0ee00182143637b71ef3,JD20220719100753718175,结构总监/经理,NaN,300800,新能源,2c92071581ebf3390182010c347d60ea,深圳市首航新能源股份有限公司,1,...,16.0,NaN,"['{""endDate"": ""2007-07"", ""unified"": true, ""deg...","['{""endDate"": ""9999-01"", ""companyName"": ""深圳市正浩...","['{""endDate"": ""2013-03"", ""companyName"": ""深圳市方格...",20220831,1,1,4.0,2022-07
82317,8a69d7c2821994530182251640042c20,8a69f68e82198e9101822012a308030b,JD20220721172428481465,HRBP,NaN,150103,HRBP,8a69f68e82198e910182200c3e497c5c,深圳微诚控股集团有限公司,1,...,4.0,NaN,"['{""endDate"": ""2018-07"", ""unified"": true, ""deg...","['{""endDate"": ""2022-03"", ""companyName"": ""奥园集团佛...","['{""endDate"": ""2022-03"", ""description"": ""工作内容:...",20220831,-1,2,4.0,2022-07
82320,8a69d7c28219945301823a2058767134,8a69f68e82198e9101823506dafb2080,JD20220725190337904313,产品经理,NaN,110100,产品经理,2c9207158114fb29018132ace5f11fe9,广州视源电子科技股份有限公司,1,...,3.0,NaN,"['{""schoolTag"": ""211"", ""endDate"": ""2019-06"", ""...","['{""endDate"": ""9999-01"", ""companyName"": ""美的"", ...","['{""name"": null}']",20220831,2,2,4.0,2022-07


In [4]:
train_data_lsa_cvjd_54339.reset_index(drop=True)

,cv_id,jd_id,jd_code,title,deadline,category_id,category_name,company_id,company_name,recruit_number,...,workYear,avatar,eduTracks,jobTracks,projectTracks,ds.1,da_label,rank_label,resume_type,order_month
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,JD20210910115615503390,市场营销负责人,2021-11-30 23:59:59.0,140101,市场营销,2c9207157bcbb76b017bcdd5f254038a,深圳市快易典教育科技有限公司,1,...,13.0,NaN,"['{""degree"": ""本科"", ""endDate"": ""2008-07"", ""id"":...","['{""companyName"": ""深圳市亿科数字科技有限公司"", ""descriptio...","['{""companyName"": ""*深圳市蜂联科技有限公司（奇虎360）"", ""desc...",20220831,3,3,NaN,2021-09
1,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,JD20211126124504034196,海外区域运营经理/高级经理,2022-02-28 23:59:59.0,130000,运营,2c9207157bf94d99017c1714f34e1715,广州市百果园网络科技有限公司,1,...,3.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2019-06"", ""id"":...","['{""companyName"": ""北京星制科技有限公司（全民快乐）"", ""descrip...","['{""description"": ""主播表现，直播平均 2 场，平台新用户 12926，主...",20220831,-1,2,4.0,2021-12
2,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,JD20211130113932574529,高级项目经理,NaN,310000,高级管理,2c9207157bf94d99017c2f902e952262,深圳纳欣科技有限公司,1,...,5.0,https://image0.lietou-static.com/big/5f8fa3bde...,"['{""degree"": ""硕士"", ""endDate"": ""2016-07"", ""id"":...","['{""companyName"": ""酷狗音乐"", ""companyTags"": ""互联网+...","['{""companyName"": ""酷狗音乐"", ""description"": ""酷狗20...",20220831,1,1,4.0,2021-12
3,2c9207157dbedf8d017dc1503c891402,2c9207157dc878e5017de5ee152b646c,JD20211223141525111469,阿里云智能-金融行业解决方案架构师（公有云）,NaN,100405,IT技术支持,2c9207157ccbcd86017d098f2b6454be,阿里云计算有限公司,1,...,11.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2010-07"", ""id"":...","['{""companyName"": ""华为技术有限公司"", ""department"": ""年...","['{""description"": ""项目职务: 项目经理\\n项目描述: 引导公司 IT ...",20220831,2,2,4.0,2021-12
4,2c9207157dbedf8d017dc1503c891402,2c9207157dbedfdb017dc14a85ff432f,JD20211216113026318551,金融行业解决方案架构师,NaN,101201,售前技术支持,2c9207157ccbcd86017d098f2b6454be,阿里云计算有限公司,2,...,11.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2010-07"", ""id"":...","['{""companyName"": ""华为技术有限公司"", ""department"": ""年...","['{""description"": ""项目职务: 项目经理\\n项目描述: 引导公司 IT ...",20220831,1,1,4.0,2021-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54334,8a69d7a68219933d018248864f066edd,8a69f601823a92b00182494187c01920,JD20220725145055775181,C++开发工程师-智能创作,NaN,100102,C++,2c9207157d95bfde017da1e665a9550d,北京字节跳动科技有限公司,1,...,9.0,NaN,"['{""schoolTag"": ""211"", ""endDate"": ""2022-01"", ""...","['{""endDate"": ""2022-07"", ""companyName"": ""北京开课吧...","['{""description"": ""项目说明: 短信平台是开课吧公司级短信需求处理的基础服...",20220831,2,2,4.0,2022-07
54335,8a69d7a68219933d01824944cb1533be,8a69c4a8820f0ee00182143637b71ef3,JD20220719100753718175,结构总监/经理,NaN,300800,新能源,2c92071581ebf3390182010c347d60ea,深圳市首航新能源股份有限公司,1,...,16.0,NaN,"['{""endDate"": ""2007-07"", ""unified"": true, ""deg...","['{""endDate"": ""9999-01"", ""companyName"": ""深圳市正浩...","['{""endDate"": ""2013-03"", ""companyName"": ""深圳市方格...",20220831,1,1,4.0,2022-07
54336,8a69d7c2821994530182251640042c20,8a69f68e82198e9101822012a308030b,JD20220721172428481465,HRBP,NaN,150103,HRBP,8a69f68e82198e910182200c3e497c5c,深圳微诚控股集团有限公司,1,...,4.0,NaN,"['{""endDate"": ""2018-07"", ""unified"": true, ""deg...","['{""endDate"": ""2022-03"", ""companyName"": ""奥园集团佛...","['{""endDate"": ""2022-03"", ""description"": ""工作内容:...",20220831,-1,2,4.0,2022-07
54337,8a69d7c28219945301823a2058767134,8a69f68e82198e9101823506dafb2080,JD20220725190337904313,产品经理,NaN,110100,产品经理,2c9207158114fb29018132ace5f11fe9,广州视源电子科技股份有限公司,1,...,3.0,NaN,"['{""schoolTag"": ""211"", ""endDate"": ""2019-06"", ""...","['{""endDate"": ""9999-01"", ""companyName"": ""美的"", ...","['{""name"": null}']",20220831,2,2,4.0,2022-07


In [5]:
pd.read_pickle('../generalization_data/train_description_tfidf_lsa.pkl')

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,0.245978,0.219388,-0.001255,0.060505,-0.019605,-0.018920,-0.069819,0.049438,-0.047138,0.038305,...,-0.008249,0.001610,-0.021326,0.023228,-0.005978,0.010373,-0.008609,-0.009598,-0.018672,-0.004692
1,0.263932,0.104354,-0.086183,-0.067204,0.090540,0.068917,-0.058438,0.039966,-0.050826,0.062841,...,-0.038992,0.014570,-0.031770,-0.002013,0.000193,0.003466,-0.000330,-0.030817,0.007213,-0.004130
2,0.276421,-0.027352,0.082342,0.081767,-0.026445,0.040150,-0.166881,-0.146737,-0.006918,0.022834,...,-0.011792,0.019838,0.058874,0.046240,0.001635,0.034112,-0.049644,-0.012443,-0.045069,-0.055091
3,0.203287,-0.005198,-0.064189,0.127611,0.017325,-0.101736,0.151930,-0.042431,-0.012835,0.016266,...,0.027772,-0.062224,-0.041545,-0.031999,-0.027054,0.015455,0.006008,0.053297,-0.020676,-0.038348
4,0.204987,-0.006300,-0.064196,0.126072,0.016448,-0.101227,0.151094,-0.042071,-0.013972,0.017931,...,0.025657,-0.063846,-0.038687,-0.031527,-0.029348,0.016352,0.003930,0.049575,-0.028572,-0.034439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54334,0.262550,-0.085161,0.077126,0.071275,-0.005347,-0.078708,0.019406,-0.024365,-0.076055,0.089592,...,-0.030109,-0.018343,0.008360,0.014661,-0.005713,-0.039473,-0.037732,-0.022184,-0.065903,0.058242
54335,0.134888,-0.043534,0.027937,0.016888,-0.039027,0.016181,-0.029961,-0.051793,0.005803,0.001074,...,0.006023,0.019793,0.022524,0.013595,0.013955,0.036088,0.008624,0.018415,-0.000551,0.002733
54336,0.192223,0.250674,0.053165,-0.120097,-0.116748,-0.052941,0.113171,0.097596,-0.007032,0.012874,...,0.062382,0.007490,-0.021115,0.015207,0.026372,-0.021530,0.014231,0.024896,-0.033106,-0.030288
54337,0.255035,-0.094685,-0.170120,-0.007026,-0.024592,-0.049753,0.043363,-0.068608,-0.007928,-0.002506,...,0.009836,0.030863,-0.019922,-0.030850,0.015589,-0.032353,0.030428,0.053683,0.000512,-0.012981


# tfidf_lsa

In [6]:
# 读取生成的5份tfidf_pca文件，并拼接
# 注意，不是join的时候，concat是通过index拼接的。所以需要将索引重置。
sample = train_data_lsa_cvjd_54339.reset_index(drop=True)

# 需要将这个向量变成一个列表
def tfidf_lsa_merge_fun(series):
    return list(series)

tfidf_lsa_file = ['train_description_tfidf_lsa.pkl', 'train_position_tfidf_lsa.pkl', 'train_requirement_tfidf_lsa.pkl', 'train_skills_tfidf_lsa.pkl', 'train_title_category_tags_tfidf_lsa.pkl']
for pkl_file in tfidf_lsa_file:
    tfidf_lsa = pd.read_pickle('../generalization_data/'+pkl_file)
    print(pkl_file)
    #print(tfidf_pca)
    col_name = pkl_file.strip('.pkl') # 把skills的s去掉了
    tfidf_lsa[col_name] = tfidf_lsa.apply(tfidf_lsa_merge_fun, axis=1)
    sample = pd.concat([sample, tfidf_lsa[[col_name]]], axis=1)

tfidf_lsa_col_names = [pkl_file.strip('.pkl') for pkl_file in tfidf_lsa_file]
sample[['cv_id', 'jd_id']+tfidf_lsa_col_names]

train_description_tfidf_lsa.pkl
train_position_tfidf_lsa.pkl
train_requirement_tfidf_lsa.pkl
train_skills_tfidf_lsa.pkl
train_title_category_tags_tfidf_lsa.pkl


,cv_id,jd_id,train_description_tfidf_lsa,train_position_tfidf_lsa,train_requirement_tfidf_lsa,train_skills_tfidf_lsa,train_title_category_tags_tfidf_lsa
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,"[0.2459783180342179, 0.21938831787537966, -0.0...","[0.007319529543557632, 0.0003354154328290771, ...","[0.033013137623400135, -0.035688917703485826, ...","[0.53765937650696, -0.14639534434021456, -0.07...","[0.139618999709323, -0.030086416810889256, -0...."
1,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,"[0.26393201542159855, 0.10435436209119428, -0....","[0.0995745047074556, -0.02088326554704094, -0....","[0.37018351050902737, -0.20255122674942938, -0...","[0.236061264433608, -0.12857925603594755, 0.13...","[0.5419245911087135, -0.18238753835825924, -0...."
2,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,"[0.27642059835889154, -0.02735173829637042, 0....","[0.043222795997538296, 0.01108309987508924, 0....","[0.28941341021190825, -0.0474103810249287, -0....","[0.00406004143441403, 0.005098830611803655, 0....","[0.09166404085822458, 0.013460082203324779, -0..."
3,2c9207157dbedf8d017dc1503c891402,2c9207157dc878e5017de5ee152b646c,"[0.20328670858820733, -0.005197850015064169, -...","[0.15026256194834306, -0.004905085247130754, -...","[0.18618173823787507, 0.031168513777247584, 0....","[0.1555934824946681, 0.15537445471422626, -0.0...","[0.08785093521473429, 0.04596513112440933, -0...."
4,2c9207157dbedf8d017dc1503c891402,2c9207157dbedfdb017dc14a85ff432f,"[0.20498743446953704, -0.006300276619096516, -...","[0.15026256194834306, -0.004905085247130754, -...","[0.19163011557219942, 0.025951225287628978, 0....","[0.1555934824946681, 0.15537445471422626, -0.0...","[0.10718268355356483, 0.05565724707601688, -0...."
...,...,...,...,...,...,...,...
54334,8a69d7a68219933d018248864f066edd,8a69f601823a92b00182494187c01920,"[0.2625502866681783, -0.08516054875935031, 0.0...","[0.23530919072669182, 0.7944767263661332, 0.01...","[0.30282726216339745, 0.16097547637426476, 0.0...","[0.27210694684652126, 0.42441791865396394, -0....","[0.08847069778780742, 0.22734187469570663, -0...."
54335,8a69d7a68219933d01824944cb1533be,8a69c4a8820f0ee00182143637b71ef3,"[0.13488778904906526, -0.043533761211661984, 0...","[0.6087412984522569, -0.1473860953415076, -0.0...","[0.15424621070300917, -0.0012759763333448448, ...","[0.2628487548869251, 0.0075222509233238725, -0...","[0.19044431984360052, -0.019572235686217573, 0..."
54336,8a69d7c2821994530182251640042c20,8a69f68e82198e9101822012a308030b,"[0.192223443587156, 0.25067413083354817, 0.053...","[0.024225991980613454, -0.0015968824512811737,...","[0.23573037593299082, -0.13812537376984757, 0....","[0.3526478953565766, -0.20899689717605643, 0.1...","[0.1171332341680559, -0.006169459729397869, -0..."
54337,8a69d7c28219945301823a2058767134,8a69f68e82198e9101823506dafb2080,"[0.2550351130980516, -0.09468529288217858, -0....","[0.8678408186646723, -0.22318129673531545, -0....","[0.19019357171237353, -0.12296867979940661, -0...","[0.27948075994260324, -0.04971016234222427, 0....","[0.3466942038817421, -0.07639258814853622, 0.3..."


In [14]:
sample[['cv_id', 'jd_id']+tfidf_lsa_col_names].to_pickle('../generalization_data/train_data_lsa_cvjd_merge_54339.pkl')

In [9]:
cvjd_lsa = sample[['cv_id', 'jd_id']+tfidf_lsa_col_names]
cvjd_lsa

,cv_id,jd_id,train_description_tfidf_lsa,train_position_tfidf_lsa,train_requirement_tfidf_lsa,train_skills_tfidf_lsa,train_title_category_tags_tfidf_lsa
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,"[0.2459783180342179, 0.21938831787537966, -0.0...","[0.007319529543557632, 0.0003354154328290771, ...","[0.033013137623400135, -0.035688917703485826, ...","[0.53765937650696, -0.14639534434021456, -0.07...","[0.139618999709323, -0.030086416810889256, -0...."
1,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,"[0.26393201542159855, 0.10435436209119428, -0....","[0.0995745047074556, -0.02088326554704094, -0....","[0.37018351050902737, -0.20255122674942938, -0...","[0.236061264433608, -0.12857925603594755, 0.13...","[0.5419245911087135, -0.18238753835825924, -0...."
2,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,"[0.27642059835889154, -0.02735173829637042, 0....","[0.043222795997538296, 0.01108309987508924, 0....","[0.28941341021190825, -0.0474103810249287, -0....","[0.00406004143441403, 0.005098830611803655, 0....","[0.09166404085822458, 0.013460082203324779, -0..."
3,2c9207157dbedf8d017dc1503c891402,2c9207157dc878e5017de5ee152b646c,"[0.20328670858820733, -0.005197850015064169, -...","[0.15026256194834306, -0.004905085247130754, -...","[0.18618173823787507, 0.031168513777247584, 0....","[0.1555934824946681, 0.15537445471422626, -0.0...","[0.08785093521473429, 0.04596513112440933, -0...."
4,2c9207157dbedf8d017dc1503c891402,2c9207157dbedfdb017dc14a85ff432f,"[0.20498743446953704, -0.006300276619096516, -...","[0.15026256194834306, -0.004905085247130754, -...","[0.19163011557219942, 0.025951225287628978, 0....","[0.1555934824946681, 0.15537445471422626, -0.0...","[0.10718268355356483, 0.05565724707601688, -0...."
...,...,...,...,...,...,...,...
54334,8a69d7a68219933d018248864f066edd,8a69f601823a92b00182494187c01920,"[0.2625502866681783, -0.08516054875935031, 0.0...","[0.23530919072669182, 0.7944767263661332, 0.01...","[0.30282726216339745, 0.16097547637426476, 0.0...","[0.27210694684652126, 0.42441791865396394, -0....","[0.08847069778780742, 0.22734187469570663, -0...."
54335,8a69d7a68219933d01824944cb1533be,8a69c4a8820f0ee00182143637b71ef3,"[0.13488778904906526, -0.043533761211661984, 0...","[0.6087412984522569, -0.1473860953415076, -0.0...","[0.15424621070300917, -0.0012759763333448448, ...","[0.2628487548869251, 0.0075222509233238725, -0...","[0.19044431984360052, -0.019572235686217573, 0..."
54336,8a69d7c2821994530182251640042c20,8a69f68e82198e9101822012a308030b,"[0.192223443587156, 0.25067413083354817, 0.053...","[0.024225991980613454, -0.0015968824512811737,...","[0.23573037593299082, -0.13812537376984757, 0....","[0.3526478953565766, -0.20899689717605643, 0.1...","[0.1171332341680559, -0.006169459729397869, -0..."
54337,8a69d7c28219945301823a2058767134,8a69f68e82198e9101823506dafb2080,"[0.2550351130980516, -0.09468529288217858, -0....","[0.8678408186646723, -0.22318129673531545, -0....","[0.19019357171237353, -0.12296867979940661, -0...","[0.27948075994260324, -0.04971016234222427, 0....","[0.3466942038817421, -0.07639258814853622, 0.3..."


# work_duration equal_job salary

In [11]:
work_duration_mean_df = pd.read_pickle('../generalization_data/train_work_duration_mean.pkl')
equal_job_df = pd.read_pickle('../generalization_data/train_equal_job.pkl')
salary_df = pd.read_pickle('../generalization_data/train_salary.pkl')

In [12]:
cvjd_lsa_work_duration_equal_job_salary = cvjd_lsa.\
    join(work_duration_mean_df.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner').\
    join(equal_job_df.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner').\
    join(salary_df.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')

cvjd_lsa_work_duration_equal_job_salary

,cv_id,jd_id,train_description_tfidf_lsa,train_position_tfidf_lsa,train_requirement_tfidf_lsa,train_skills_tfidf_lsa,train_title_category_tags_tfidf_lsa,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,"[0.2459783180342179, 0.21938831787537966, -0.0...","[0.007319529543557632, 0.0003354154328290771, ...","[0.033013137623400135, -0.035688917703485826, ...","[0.53765937650696, -0.14639534434021456, -0.07...","[0.139618999709323, -0.030086416810889256, -0....",1209.5,3,-1,-1
1,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,"[0.26393201542159855, 0.10435436209119428, -0....","[0.0995745047074556, -0.02088326554704094, -0....","[0.37018351050902737, -0.20255122674942938, -0...","[0.236061264433608, -0.12857925603594755, 0.13...","[0.5419245911087135, -0.18238753835825924, -0....",76.5,3,-1,-1
2,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,"[0.27642059835889154, -0.02735173829637042, 0....","[0.043222795997538296, 0.01108309987508924, 0....","[0.28941341021190825, -0.0474103810249287, -0....","[0.00406004143441403, 0.005098830611803655, 0....","[0.09166404085822458, 0.013460082203324779, -0...",1034.0,1,450,-1
3,2c9207157dbedf8d017dc1503c891402,2c9207157dc878e5017de5ee152b646c,"[0.20328670858820733, -0.005197850015064169, -...","[0.15026256194834306, -0.004905085247130754, -...","[0.18618173823787507, 0.031168513777247584, 0....","[0.1555934824946681, 0.15537445471422626, -0.0...","[0.08785093521473429, 0.04596513112440933, -0....",NaN,2,-1,-1
4,2c9207157dbedf8d017dc1503c891402,2c9207157dbedfdb017dc14a85ff432f,"[0.20498743446953704, -0.006300276619096516, -...","[0.15026256194834306, -0.004905085247130754, -...","[0.19163011557219942, 0.025951225287628978, 0....","[0.1555934824946681, 0.15537445471422626, -0.0...","[0.10718268355356483, 0.05565724707601688, -0....",NaN,2,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...
54334,8a69d7a68219933d018248864f066edd,8a69f601823a92b00182494187c01920,"[0.2625502866681783, -0.08516054875935031, 0.0...","[0.23530919072669182, 0.7944767263661332, 0.01...","[0.30282726216339745, 0.16097547637426476, 0.0...","[0.27210694684652126, 0.42441791865396394, -0....","[0.08847069778780742, 0.22734187469570663, -0....",1075.0,5,-1,300
54335,8a69d7a68219933d01824944cb1533be,8a69c4a8820f0ee00182143637b71ef3,"[0.13488778904906526, -0.043533761211661984, 0...","[0.6087412984522569, -0.1473860953415076, -0.0...","[0.15424621070300917, -0.0012759763333448448, ...","[0.2628487548869251, 0.0075222509233238725, -0...","[0.19044431984360052, -0.019572235686217573, 0...",796.5,7,1020,850
54336,8a69d7c2821994530182251640042c20,8a69f68e82198e9101822012a308030b,"[0.192223443587156, 0.25067413083354817, 0.053...","[0.024225991980613454, -0.0015968824512811737,...","[0.23573037593299082, -0.13812537376984757, 0....","[0.3526478953565766, -0.20899689717605643, 0.1...","[0.1171332341680559, -0.006169459729397869, -0...",1019.0,1,600,120
54337,8a69d7c28219945301823a2058767134,8a69f68e82198e9101823506dafb2080,"[0.2550351130980516, -0.09468529288217858, -0....","[0.8678408186646723, -0.22318129673531545, -0....","[0.19019357171237353, -0.12296867979940661, -0...","[0.27948075994260324, -0.04971016234222427, 0....","[0.3466942038817421, -0.07639258814853622, 0.3...",NaN,4,280,240


# new feature

In [13]:
# 保存为new feature
cvjd_lsa_work_duration_equal_job_salary.to_pickle('../generalization_data/train_new_feature_54339.pkl')


# raw feature

In [14]:
# 这是用来生成lsa向量的训练集，必须用这个数据集来拼接lsa向量
# 这份文件其实是用raw_cvjd的数据和train id进行join得到的
train_data_lsa_cvjd_54339 = pd.read_pickle('../generalization_data/train_data_lsa_cvjd_54339.pkl')
train_data_lsa_cvjd_54339

,cv_id,jd_id,jd_code,title,deadline,category_id,category_name,company_id,company_name,recruit_number,...,workYear,avatar,eduTracks,jobTracks,projectTracks,ds.1,da_label,rank_label,resume_type,order_month
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,JD20210910115615503390,市场营销负责人,2021-11-30 23:59:59.0,140101,市场营销,2c9207157bcbb76b017bcdd5f254038a,深圳市快易典教育科技有限公司,1,...,13.0,NaN,"['{""degree"": ""本科"", ""endDate"": ""2008-07"", ""id"":...","['{""companyName"": ""深圳市亿科数字科技有限公司"", ""descriptio...","['{""companyName"": ""*深圳市蜂联科技有限公司（奇虎360）"", ""desc...",20220831,3,3,NaN,2021-09
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,JD20211126124504034196,海外区域运营经理/高级经理,2022-02-28 23:59:59.0,130000,运营,2c9207157bf94d99017c1714f34e1715,广州市百果园网络科技有限公司,1,...,3.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2019-06"", ""id"":...","['{""companyName"": ""北京星制科技有限公司（全民快乐）"", ""descrip...","['{""description"": ""主播表现，直播平均 2 场，平台新用户 12926，主...",20220831,-1,2,4.0,2021-12
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,JD20211130113932574529,高级项目经理,NaN,310000,高级管理,2c9207157bf94d99017c2f902e952262,深圳纳欣科技有限公司,1,...,5.0,https://image0.lietou-static.com/big/5f8fa3bde...,"['{""degree"": ""硕士"", ""endDate"": ""2016-07"", ""id"":...","['{""companyName"": ""酷狗音乐"", ""companyTags"": ""互联网+...","['{""companyName"": ""酷狗音乐"", ""description"": ""酷狗20...",20220831,1,1,4.0,2021-12
5,2c9207157dbedf8d017dc1503c891402,2c9207157dc878e5017de5ee152b646c,JD20211223141525111469,阿里云智能-金融行业解决方案架构师（公有云）,NaN,100405,IT技术支持,2c9207157ccbcd86017d098f2b6454be,阿里云计算有限公司,1,...,11.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2010-07"", ""id"":...","['{""companyName"": ""华为技术有限公司"", ""department"": ""年...","['{""description"": ""项目职务: 项目经理\\n项目描述: 引导公司 IT ...",20220831,2,2,4.0,2021-12
6,2c9207157dbedf8d017dc1503c891402,2c9207157dbedfdb017dc14a85ff432f,JD20211216113026318551,金融行业解决方案架构师,NaN,101201,售前技术支持,2c9207157ccbcd86017d098f2b6454be,阿里云计算有限公司,2,...,11.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2010-07"", ""id"":...","['{""companyName"": ""华为技术有限公司"", ""department"": ""年...","['{""description"": ""项目职务: 项目经理\\n项目描述: 引导公司 IT ...",20220831,1,1,4.0,2021-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82315,8a69d7a68219933d018248864f066edd,8a69f601823a92b00182494187c01920,JD20220725145055775181,C++开发工程师-智能创作,NaN,100102,C++,2c9207157d95bfde017da1e665a9550d,北京字节跳动科技有限公司,1,...,9.0,NaN,"['{""schoolTag"": ""211"", ""endDate"": ""2022-01"", ""...","['{""endDate"": ""2022-07"", ""companyName"": ""北京开课吧...","['{""description"": ""项目说明: 短信平台是开课吧公司级短信需求处理的基础服...",20220831,2,2,4.0,2022-07
82316,8a69d7a68219933d01824944cb1533be,8a69c4a8820f0ee00182143637b71ef3,JD20220719100753718175,结构总监/经理,NaN,300800,新能源,2c92071581ebf3390182010c347d60ea,深圳市首航新能源股份有限公司,1,...,16.0,NaN,"['{""endDate"": ""2007-07"", ""unified"": true, ""deg...","['{""endDate"": ""9999-01"", ""companyName"": ""深圳市正浩...","['{""endDate"": ""2013-03"", ""companyName"": ""深圳市方格...",20220831,1,1,4.0,2022-07
82317,8a69d7c2821994530182251640042c20,8a69f68e82198e9101822012a308030b,JD20220721172428481465,HRBP,NaN,150103,HRBP,8a69f68e82198e910182200c3e497c5c,深圳微诚控股集团有限公司,1,...,4.0,NaN,"['{""endDate"": ""2018-07"", ""unified"": true, ""deg...","['{""endDate"": ""2022-03"", ""companyName"": ""奥园集团佛...","['{""endDate"": ""2022-03"", ""description"": ""工作内容:...",20220831,-1,2,4.0,2022-07
82320,8a69d7c28219945301823a2058767134,8a69f68e82198e9101823506dafb2080,JD20220725190337904313,产品经理,NaN,110100,产品经理,2c9207158114fb29018132ace5f11fe9,广州视源电子科技股份有限公司,1,...,3.0,NaN,"['{""schoolTag"": ""211"", ""endDate"": ""2019-06"", ""...","['{""endDate"": ""9999-01"", ""companyName"": ""美的"", ...","['{""name"": null}']",20220831,2,2,4.0,2022-07


In [ ]:
# 这些特征在原始的cv和jd表里
raw_columns = ['age', 'recruit_number', 'workYear', 'min_month_salary', 'max_month_salary', 'pay_months']

# 将raw特征保存
train_data_lsa_cvjd_54339[['cv_id', 'jd_id'] + raw_columns].to_pickle()
